# This is the code which solves the Static Traffic Assignment Problem

We want to solve the following problem (TAP-C):
\begin{align}
\min_{f, h} &\sum_{a} \int_{0}^{f_a} c_a(s)\; \text{d}s
\\
\text{s.t.  } & \;\; f = \Delta h
\\
& \;\; 
h \geq 0
\\
& \;\; 
A h = d
\\
& \;\; 
f \leq u
\end{align}

This problem is hard to compute, even if it is a convex problem. Computing all the path of a network is NP-hard (with respect to the number of edges and links).

Nevertheless, the TAP can be solve using a Frank-Wolf algorithm:
\begin{align}
&\text{1. TO DO }
\\
&\text{2. }
\end{align}


#### Remarks
First, we tried to solve the STA with CVX.
This does not work because the values of the travel time are to big. 

Then, we solves the STA using links flow. 
This does not help us in our project because we need the path flow to compute the dual of the TAP-C.

Finally, we solve the STA using paths flow.
The issue here is that finding every path in a network in a NP-complete problem.
But if we can compute every path (we only need to do it one time), then it is really fast to compute the shortest path. We do not need to do a Dikjstra's algorithm to compute the shortest path, only a big matrix multiplication and a array sorting are enough. This might be more difficult on a laptop. But it might be faster on a HPC.

An other possibility, would be to compute the paths during the Frank Wolf's algorithm. THIS HAS TO BE DONE.

### I. We load and clean the network and the demand data

In [1]:
import numpy as np
import scipy
import scipy.sparse

In [2]:
I210 = 'data/I210'
Chic = 'data/Chicago'
Anah = 'data/Anaheim'
Siou = 'data/SiouxFalls'
Brae = 'data/braess'

network = I210
debug = True

In [3]:
graph = np.loadtxt(network + '_net.csv', delimiter=',', skiprows=1)
demand = np.loadtxt(network + '_od.csv', delimiter=',', skiprows=1)

In [4]:
"""
We need to demand to be an array like:
[
[o1, d1, demand from o1 to d1],
...,
[on, dn, demand from on to dn]
]

We need the graph to be an array like:
[
[id link 1, node origin link 1, node destination link 1, a1, a2, a3, a4],
...,
[id link n, node origin link n, node destination link n, a1, a2, a3, a4]
]
where the node are indexed from 0 to nb_nodes-1,
    the links are indexed from 0 to nb_links-1

One can add some checks here to be sure that demand and graph respect this format!
"""

def cleaning_input(graph, demand):
    # in the case where there is only one o-d, then demand is interpret as a single row and not as a matrix (2d array)
    try:
        demand.shape[1]
    except:
        demand = np.array([demand])
    nb_ods = int(demand.shape[0])

    # in the case where the index of the od pairs does not begin by 0, we rename the od pairs
    first_index_od = min(np.min(graph[:,1]), np.min(graph[:,2]))
    graph[:,1] = graph[:,1]-first_index_od
    graph[:,2] = graph[:,2]-first_index_od
    demand[:,0] = demand[:,0] - first_index_od
    demand[:,1] = demand[:,1] - first_index_od
    return graph, demand

graph, demand = cleaning_input(graph, demand)
nb_links = int(np.max(graph[:,0])+1)
nb_nodes = int(max(np.max(graph[:,1]), np.max(graph[:,2]))+1)
nb_ods = int(demand.shape[0])
if debug:
    print("nb nodes = " + str(nb_nodes))
    print("nb links = " + str(nb_links))
    print("nb ods = " + str(nb_ods))

nb nodes = 20
nb links = 41
nb ods = 1


### 2. We compute the incidence matrix

In [5]:
"""
We need to define 2 matrix called path_matrix (or delta) and route2od which gives are:
- delta is the incidence matrix. Each row of delta is a path, each column a link.
    delta[i][j] is 1 if the link j is used by the path i, 0 otherwise
- route2od is the array which recover the origin and destination of every path
    route2od[i] is the indice of the line which codes the origin and the demand
"""

from collections import defaultdict 
   
# This class represents a directed graph  
# using adjacency list representation 
class Graph: 
   
    def __init__(self,vertices): 
        #No. of vertices 
        self.V= vertices  
        # default dictionary to store graph 
        self.graph = defaultdict(list)  
   
    # function to add an edge to graph 
    def addEdge(self,u,v): 
        self.graph[u].append(v) 
   
    def printAllPathsUtil(self, u, d, visited, path, graph_dict): 
        # Mark the current node as visited and store in path 
        visited[u]= True
        path.append(u) 
        
        if u == d: 
            path_tmp = np.zeros(shape=(self.V))
            for i in range(len(path)-1):
                # link_indice = list(G.todok().keys()).index((path[i], path[i+1]))
                link_indice = graph_dict[path[i]][path[i+1]]
                path_tmp[link_indice] = 1
            if self.paths_m.shape[0]==0:
                self.paths_m = path_tmp.reshape((1, self.V))
            else:
                self.paths_m = np.append(self.paths_m, path_tmp.reshape((1, self.V)), axis=0)
        else: 
            # If current vertex is not destination 
            # Recur for all the vertices adjacent to this vertex 
            for i in self.graph[u]: 
                if visited[i]==False: 
                    self.printAllPathsUtil(i, d, visited, path, graph_dict) 
        # Remove current vertex from path[] and mark it as unvisited 
        path.pop()
        visited[u]= False
   
   
    # Prints all paths from 's' to 'd' 
    def printAllPaths(self,s, d, graph_dict): 
        self.paths_m = np.array([])
        # Mark all the vertices as not visited 
        visited =[False]*(self.V)
  
        # Create an array to store paths 
        path = [] 
        
        # Call the recursive helper function to print all paths 
        self.printAllPathsUtil(s, d, visited, path, graph_dict) 
        return self.paths_m 

def return_graph_dict(graph):
    # graph_dict gives the line of the graph matrix corresponding to the destination d and the origin o
    graph_dict = {}
    for i in range(graph.shape[0]):
        try: 
            graph_dict[int(graph[i][1])]
        except:
            graph_dict[int(graph[i][1])] = {}
        graph_dict[int(graph[i][1])][int(graph[i][2])] = int(graph[i][0])
    return graph_dict
    
# argument graph
def delta_matrix(graph, demand):
    route2od = np.array([])
    paths_matrix = np.array([])
    graph_dict = return_graph_dict(graph)
    g = Graph(graph.shape[0]) 
    for line in graph:
        g.addEdge(int(line[1]), int(line[2]))

    for i in range(demand.shape[0]):
        s = int(demand[i][0]) ; d = int(demand[i][1])
        paths_matrix_tmp = g.printAllPaths(s, d, graph_dict)
        route2od_tmp = np.ones(paths_matrix_tmp.shape[0])
        route2od_tmp = route2od_tmp*i
        if paths_matrix.shape[0]==0:
            paths_matrix = paths_matrix_tmp
        else:
            paths_matrix = np.append(paths_matrix, paths_matrix_tmp, axis=0)
        route2od = np.append(route2od, route2od_tmp)
    return paths_matrix, route2od

### 3. We define the travel time function. We encode the capacity here.

In [6]:
import sys
max_float = 1e+10
if debug:
    print(max_float)

# TO DO ADD THE CAPACITY
def travel_time(f, c = -1):
    if c == -1:
        c = [max_float for i in range(len(f))] # this might be to much, to do once we have everything done
    # here we need to have the same indexation for graph and f.
    tt_tmp = graph[:,3] + graph[:,4]*f + graph[:,5]*(f**2) + graph[:,6]*(f**3) + graph[:,7]*(f**4)
    tt_tmp = [tt_tmp[i] if f[i]<c[i] else max_float for i in range(len(f))]
    return tt_tmp

10000000000.0


In [7]:
def update_travel_time(tt, G):
    for i in range(graph.shape[0]):
        G[int(graph[i][1]),int(graph[i][2])] = tt[i]
    return G

In [8]:
G = np.zeros(shape=(nb_nodes,nb_nodes))
G = update_travel_time(travel_time(np.zeros(nb_links)), G) # , [1,1,-1,1,1]))
# the following line make the matrix sparse, one can use G.toarray() to do the opposite
G = scipy.sparse.csr_matrix(G)

In [9]:
delta, route2od = delta_matrix(graph, demand)
if debug:
    print(delta.shape)
    print(delta[0])

(243, 41)
[1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]


In [10]:
tt = travel_time(np.zeros(nb_links))
delta_csr = scipy.sparse.csr_matrix(delta)
if debug:
    print((delta_csr @ tt))
    print(np.argmin(delta_csr @ tt))

[10.7   9.17 11.59 11.96  8.35 10.77 13.76 10.15 11.57 11.78 10.25 12.67
 10.94  7.33  9.75 12.74  9.13 10.55 13.74 12.21 14.63 12.9   9.29 11.71
 13.74 10.13 11.55 12.07 10.54 12.96 13.33  9.72 12.14 15.13 11.52 12.94
 10.89  9.36 11.78 10.05  6.44  8.86 11.85  8.24  9.66 12.85 11.32 13.74
 12.01  8.4  10.82 12.85  9.24 10.66 13.74 12.21 14.63 15.   11.39 13.81
 16.8  13.19 14.61 12.56 11.03 13.45 11.72  8.11 10.53 13.52  9.91 11.33
 13.76 12.23 14.65 12.92  9.31 11.73 13.76 10.15 11.57  9.79  8.26 10.68
 11.05  7.44  9.86 12.85  9.24 10.66 10.87  9.34 11.76 10.03  6.42  8.84
 11.83  8.22  9.64 12.83 11.3  13.72 11.99  8.38 10.8  12.83  9.22 10.64
  8.88  7.35  9.77 10.14  6.53  8.95 11.94  8.33  9.75  7.7   6.17  8.59
  6.86  3.25  5.67  8.66  5.05  6.47  9.66  8.13 10.55  8.82  5.21  7.63
  9.66  6.05  7.47 10.55  9.02 11.44 11.81  8.2  10.62 13.61 10.   11.42
  9.37  7.84 10.26  8.53  4.92  7.34 10.33  6.72  8.14 10.57  9.04 11.46
  9.73  6.12  8.54 10.57  6.96  8.38 11.44  9.91 12

### 3. We upload the flow allocation 
#### 3.1 We compute the travel time of each path. 
#### 3.2 We compute the all or nothing allocation by putting all the demand on shortest paths.
#### 3.3 We update the flow allocation with a bisection.

In [26]:
"""
The function potential is used to compute the line search between 
the all or nothing allocation and the current flow allocation
The function potential returns the objective function corresponding to
the flow allocation f.
"""

def potential(graph, f, c=-1):
    # this routine is useful for doing a line search
    # computes the potential at flow assignment f
    if c == -1:
        c = [max_float for i in range(len(f))] # this might be to much, to do once we have everything done
    # here we need to have the same indexation for graph and f.
    pot_tmp = graph[:,3]*f + 1/2*graph[:,4]*(f**2) + 1/3*graph[:,5]*(f**3) + 1/4*graph[:,6]*(f**4) + 1/5*graph[:,7]*(f**5)
    # TO DO check the following line
    # pot_tmp = [pot_tmp[i] if f[i]<c[i] else f[i]*max_float for i in range(len(f))]
    return np.sum(pot_tmp)

    # return np.sum(graph[:,3]*f + 1/2*graph[:,4]*(f**2) + 1/3*graph[:,5]*(f**3) + 1/4*graph[:,6]*(f**4) + 1/5*graph[:,7]*(f**5))
    #links = int(np.max(graph[:, 0]) + 1)
    #g = np.copy(
    #    graph.dot(np.diag([1., 1., 1., 1., 1 / 2., 1 / 3., 1 / 4., 1 / 5.])))
    #x = np.power(f.reshape((links, 1)), np.array([1, 2, 3, 4, 5]))
    #return np.sum(np.einsum('ij,ij->i', x, g[:, 3:]))


def line_search(f, res=20):
    debug_local = False
    # on a grid of 2^res points bw 0 and 1, find global minimum
    # of continuous convex function
    # here we do a bisection
    d = 1. / (2**res - 1)
    l, r = 0, 2**res - 1
    while r - l > 1:
        if f(l * d) <= f(l * d + d):
            return l * d
        if f(r * d - d) >= f(r * d):
            return r * d
        # otherwise f(l) > f(l+d) and f(r-d) < f(r)
        m1, m2 = (l + r) / 2, 1 + (l + r) / 2
        if debug_local:
            print(l * d, end=": ")
            print(f(l * d))
            print(r * d, end=": ")
            print(f(r * d))
            print(str(m1 * d) + " = " + str(f(m1 * d)))
            print(str(m2 * d) + " = " + str(f(m2 * d)))
            print()
        if f(m1 * d) < f(m2 * d):
            r = m1
        if f(m1 * d) > f(m2 * d):
            l = m2
        if f(m1 * d) == f(m2 * d):
            return m1 * d
    return l * d

In [27]:
# check sparse vector
h = scipy.sparse.lil_matrix(np.zeros(delta_csr.shape[0]))
# the following is for one od
# demand[0][2] = 10
# TO DO MODIFY TO TAKE INTO ACCOUNT SEVERALS OD
h[:,0] = demand[0][2]
print(h)

  (0, 0)	25000.0


In [28]:
def compute_initial_flow(demand):
    return h
def solver(eps, nb_iter, demand, delta_csr, c=-1):
    h = compute_initial_flow(demand)
    for i in range(nb_iter):
        f = h @ delta_csr
        tt_flow = travel_time(f.toarray()[0], c)
        tt_p = delta_csr @ tt_flow
        # the following is for one od
        p_aon = np.argmin(tt_p)
        # the following is for one od
        # TO MODIFY TO TAKE INTO ACCOUNT SEVERAL ODs
        h_aon = scipy.sparse.lil_matrix(np.zeros(delta_csr.shape[0]))
        h_aon[:, p_aon] = demand[0][2]
        f_aon = h_aon @ delta_csr
        t = potential(graph, f.toarray()[0])
        s = line_search(lambda a: potential(graph, (1. - a) * f.toarray()[0] + a * f_aon.toarray()[0], c))
        h = (1.-s)*h + s*h_aon
        if s < eps:
            break
        if i % (nb_iter/20) == 0:
            print(i)
        if debug:
            print(f.toarray()[0])
            print(tt_flow)
            print(tt_p)
            print()
            print(potential(graph,f.toarray()[0])-potential(graph, (1. - s) * f.toarray()[0] + s * f_aon.toarray()))
            print(s)
    if debug:
        print(h)
    return h

In [29]:
eps=1e-10
nb_iter = 1000
# c = [100, 100, 2, 100, 100]

h = solver(eps, nb_iter, demand, delta_csr) #, c)

f = h @ delta_csr
tt_flow = travel_time(f.toarray()[0]) #, c)
tt_p = delta_csr @ tt_flow

0
[25000.     0. 25000.     0. 25000.     0. 25000.     0. 25000.     0.
 25000. 25000.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0. 25000.
 25000.]
[1.29375e+18, 1.15, 1.29375e+18, 1.14, 1.575e+18, 1.13, 1.18125e+18, 1.05, 2.1093750000000003e+18, 1.04, 1.021875e+18, 1.0781249999999999e+18, 0.48, 0.36, 1.14, 0.48, 0.39, 1.13, 0.58, 0.38, 1.05, 0.43, 0.48, 1.04, 0.77, 0.5, 1.09, 0.47, 0.36, 1.38, 0.39, 1.38, 0.38, 1.68, 0.48, 1.25, 0.5, 2.22, 0.47, 330000000000000.25, 435000000000000.3]
[9.553890e+18 6.422640e+18 6.422640e+18 8.372640e+18 5.241390e+18
 5.241390e+18 8.372640e+18 5.241390e+18 5.241390e+18 7.978890e+18
 4.847640e+18 4.847640e+18 6.797640e+18 3.666390e+18 3.666390e+18
 6.797640e+18 3.666390e+18 3.666390e+18 7.978890e+18 4.847640e+18
 4.847640e+18 6.797640e+18 3.666390e+18 3.666390e+18 6.797640e+18
 3.666390e+18 3.666390e+18 8.2601

 2.22727394e+15 2.22736438e+15 2.22783908e+15]

25865990670336.0
0.0003666877751129509
[ 2657.74207204     0.          2941.40568707     0.
  2938.11631893    49.58632099  2931.68545569    52.77122028
  2704.43172263   263.08077752  2704.43172263  2657.74207204
 19513.6203616   2828.63756635   283.66361503 19121.58057233
   149.07313464    46.29695285 19124.64706707    60.48947599
    46.34035704 19131.33736856    30.99782493    35.82704447
 19449.50321306    55.12710905     0.             0.
     0.          2828.63756635    40.6969604   2937.0137406
    60.26660259  2937.23661401    31.25726318  2936.97717575
   146.0392205   2846.06506431  2846.06506431 25000.
 25000.        ]
[165250070509796.75, 1.15, 247919130000224.75, 1.14, 300466781392656.1, 16395975.655388745, 223383595367628.62, 19542904.58316051, 288866943292997.75, 11956416850.927073, 139939985861941.12, 137708392091497.28, 267253786944662.25, 55312458444812.266, 17714570462.573597, 246415267755051.9, 462246236.5460743, 12

[ 2594.8002703      0.          2936.16317714     0.
  2933.0505408     74.09500728  2926.47505243    88.44269761
  2695.28325959   302.71870444  2695.28325959  2594.8002703
 19568.62140197  2836.57832773   341.36290684 19126.06513223
   170.62029402    70.98237094 19133.21355556    79.9555776
    81.86720923 19142.58824652    56.46734195    71.5269116
 19458.25830927    99.2299238      0.             0.
     0.          2836.57832773    69.42693111  2937.77169063
    83.99136459  2933.73590364    59.26654454  2930.93670105
   183.70819371  2846.45843115  2846.45843115 25000.
 25000.        ]
[150143337722734.78, 1.15, 246156370278970.12, 1.14, 298399925081300.5, 81742039.86289409, 221799770082961.0, 154187251.2260831, 284978053293603.9, 20960498405.891644, 138056034706679.22, 125119448102278.98, 270279673429222.78, 55936187736563.43, 37151932179.82786, 246646514923794.78, 793229017.5848105, 68848058.56785558, 298477015562394.44, 37272618.370397665, 113198452.25497472, 221718693776911.

[ 2589.76291535     0.          2937.55856619     0.
  2933.71810072    78.12428135  2926.06176023    91.60427461
  2696.58382044   305.24937282  2696.58382044  2589.76291535
 19573.1598382   2837.07724646   347.79565084 19124.93115889
   173.63631187    74.28381588 19132.95848073    82.57661205
    83.94793411 19141.94105619    62.32170508    75.77143303
 19456.96492032   103.76961644     0.             0.
     0.          2837.07724646    73.20328341  2937.51027492
    86.76346842  2933.32341855    63.64794005  2931.99718359
   189.31554078  2846.45125925  2846.45125925 25000.
 25000.        ]
[148980819385847.56, 1.15, 246624639673069.38, 1.14, 298671680162271.8, 101026156.69813056, 221674501701288.97, 177444892.48275477, 285528495842918.44, 21670239052.705246, 138322693541680.5, 124150682821539.64, 270530498223739.75, 55975552030160.2, 40032497073.57186, 246588026009079.12, 850820723.293266, 82578408.90098026, 298461099270817.5, 42405587.560605265, 125152807.499061, 221688711014655

[ 2587.54138123     0.          2937.4339679      0.
  2933.24824446    79.26100815  2926.05119367    92.13328806
  2696.54378088   306.10157196  2696.54378088  2587.54138123
 19574.68982782  2837.76879095   349.89258667 19125.06585278
   175.28826413    75.07528471 19133.18580681    84.03730079
    84.93623728 19141.71093095    63.32558347    76.59415917
 19456.89053041   105.19982891     0.             0.
     0.          2837.76879095    75.55687576  2937.50017932
    87.97153138  2933.56594873    64.65365682  2932.23787538
   190.87201558  2846.56568871  2846.56568871 25000.
 25000.        ]
[148470285616011.4, 1.15, 246582799414617.34, 1.14, 298480388369649.5, 107035553.14981745, 221671299686850.9, 181579504.09617186, 285511537823348.56, 21913251369.070004, 138314478323311.06, 123725238013342.86, 270615095170331.44, 56030148697199.94, 41006722286.600685, 246594972805285.7, 883664096.7942023, 86154435.74479467, 298475284050665.06, 45486573.33216741, 131151304.48419097, 221678050601

[ 2585.81375257     0.          2937.31986575     0.
  2933.24418238    80.84065285  2926.14735155    92.82252833
  2696.47139817   306.39774404  2696.47139817  2585.81375257
 19576.56020759  2837.62603984   351.50611318 19125.13915407
   176.89320281    76.76496949 19133.45622069    85.45892243
    85.72569749 19141.87427039    63.93284558    76.72179066
 19457.07583043   105.91016894     0.             0.
     0.          2837.62603984    76.97826247  2937.54098018
    89.70030569  2933.29959692    65.25406443  2931.97837807
   191.43577561  2846.4527714   2846.4527714  25000.
 25000.        ]
[148074164807020.4, 1.15, 246544488440713.16, 1.14, 298478734982690.8, 115826767.98260158, 221700439971550.56, 187074295.3373977, 285480883374356.7, 21998184204.304867, 138299627945799.47, 123395137339183.69, 270718540089011.22, 56018875386137.16, 41768379165.88299, 246598753359050.38, 916474593.878164, 94176385.24406968, 298492158098423.5, 48643463.670207955, 136095778.31649014, 22168561716331

[ 2584.96957304     0.          2937.29655495     0.
  2933.21625225    81.18972179  2926.0264103     93.28088418
  2696.4373341    306.6076124   2696.4373341   2584.96957304
 19577.46068498  2837.56974198   352.32698191 19125.26926768
   177.43735886    77.10941909 19133.59554394    86.02694897
    86.09104223 19142.08829415    64.60919776    77.0185362
 19457.22866227   106.51921053     0.             0.
     0.          2837.56974198    77.57292346  2937.43417737
    90.27292252  2933.18820381    65.91210602  2931.88529555
   192.07050246  2846.33400363  2846.33400363 25000.
 25000.        ]
[147880894925251.78, 1.15, 246536662148826.47, 1.14, 298467366774880.56, 117840317.05666108, 221663789642850.2, 190796831.82241666, 285466457919967.4, 22058517134.102722, 138292639614561.98, 123234079104376.5, 270768353282520.62, 56014429910980.695, 42159913141.0283, 246605464145607.2, 927803713.7381762, 95878099.25310972, 298500852262276.8, 49949704.52732311, 138430697.10582918, 221695531926021

In [30]:
if debug:
    print(h)
    print(tt_p[h.toarray()[0]!=0])
    print(tt_flow)
    print(delta_csr @ tt_flow)
    print(np.argsort(delta_csr @ tt_flow))
    print(h.toarray()[0])

  (0, 0)	2584.7168984300833
  (0, 82)	178.92607333060076
  (0, 83)	46.119723337645055
  (0, 85)	23.785854606239635
  (0, 87)	8.282834989250826
  (0, 88)	0.04768107833735472
  (0, 90)	3.072346434406515
  (0, 91)	2.311973865412751
  (0, 93)	0.2798252675103561
  (0, 94)	7.711722544189058
  (0, 95)	0.45233033247967436
  (0, 97)	4.458684884180674
  (0, 103)	18.180308669095382
  (0, 107)	7.934704265716068
  (0, 109)	0.4047628275507175
  (0, 110)	0.19056507834119327
  (0, 111)	9.6953200890968
  (0, 112)	12.818897733100254
  (0, 113)	0.09534183247621568
  (0, 114)	0.7346830968426813
  (0, 115)	8.53340508992268
  (0, 117)	0.4289339177776376
  (0, 118)	32.84897346951361
  (0, 119)	0.1906377013293551
  (0, 120)	18.266798116025534
  :	:
  (0, 175)	3.6449814730771206
  (0, 178)	1.9301833615815627
  (0, 180)	0.39316971399606665
  (0, 182)	12.218117569697586
  (0, 188)	18.831844802387025
  (0, 195)	0.07151235587223939
  (0, 196)	0.047677469079976706
  (0, 199)	2.6416962162612534
  (0, 200)	0.11913239

In [40]:
g = h.toarray()[0]!=0
print(np.sum(g*1.))

81.0
